In [ ]:
# ! pip install Bio

In [1]:
from Bio import pairwise2
import random
import string
def check_bio_stats(X,Y, alpha):
    X = X.replace('_','-')
    Y = Y.replace('_','-')
    bio_dict = {}
    for k,v in alpha.items():
        for k1,v1 in v.items():
            bio_dict[(k,k1)] = -v1
    #print(bio_dict)

    alignments = pairwise2.align.globalds(X, Y,bio_dict,-30,-30)
    return -1 * alignments[0].score


In [2]:
def get_random_string(length):
    letters = 'ACGT'
    result_str = ''.join(random.choice(letters) for i in range(length))
    return result_str

In [3]:
def read_and_generate_strings(input_file):
    X=''
    Y=''
    with open(f'{input_file}') as file:
        lines = file.readlines()
        X = lines[0].strip()
        Y = ''
        flag = 0

        for i in range(1,len(lines)):
            val = lines[i].strip()
            try:
                val = int(val)

                if flag == 0:
                    X = X[:val+1] + X + X[val+1:]
                else:
                    Y = Y[:val+1] + Y + Y[val+1:]
            except:
                Y = val
                flag = 1
    return X,Y

def check_end_case_and_add_gaps(x,y):
    if x=="" or y=="":
        if x=="":
            return True,"_" * len(y), y
        elif y=="":
            return True, x, "_" * len(x)
        else:
            return True, "",""
    elif len(x)==1 and len(y)==1:
        return True, x,y
    return False, x, y

def compute_cost(X_align, Y_align, alpha, delta):
    i = 0
    cost = 0
    while i<len(X_align):
        if X_align[i] != '_' and Y_align[i] != '_':
            cost+= alpha[X_align[i]][Y_align[i]]
        else:
            cost+= delta
        i+=1
    return cost

class SequenceAlignmentBasic():
    
    def __init__(self,X,Y, alpha, delta):
        self.X = X
        self.Y = Y
        self.alpha = alpha
        self.delta = delta
        self.len_x = len(self.X)
        self.len_y = len(self.Y)
        self.dp = [[0 for j in range(self.len_y+1)] for i in range(self.len_x+1)]
        
    def calculate_alignment_cost(self):       

        for i in range(self.len_x+1):
            self.dp[i][0] = i*delta   

        for i in range(self.len_y+1):
            self.dp[0][i] = i*delta

        for i in range(1,self.len_x+1):
            for j in range(1,self.len_y+1):

                matching = self.dp[i-1][j-1] + self.alpha[self.X[i-1]][self.Y[j-1]]
                mismatch_x = self.dp[i-1][j] + self.delta
                mismatch_y = self.dp[i][j-1] + self.delta

                self.dp[i][j] = min(matching,mismatch_x,mismatch_y)

        return self.dp[self.len_x-1][self.len_y-1]

    def find_alignment(self):
        X_aligned = ""
        Y_aligned = ""
        i,j = self.len_x-1, self.len_y-1
        while i >= 0 and j >= 0:
            
            matching = self.dp[i][j] + self.alpha[self.X[i]][self.Y[j]]
            mismatch_x = self.dp[i][j+1] + self.delta
            mismatch_y = self.dp[i+1][j] + self.delta
            
            min_cost = min(matching,mismatch_x,mismatch_y)
            
            if min_cost==matching:
                X_aligned = self.X[i] + X_aligned
                Y_aligned = self.Y[j] + Y_aligned
                i-=1
                j-=1
            elif min_cost== mismatch_x:
                X_aligned = self.X[i] + X_aligned
                Y_aligned = '_' + Y_aligned
                i-=1
            elif min_cost== mismatch_y:
                X_aligned = '_' + X_aligned
                Y_aligned = self.Y[j] + Y_aligned
                j-=1

        while i >= 0:
            X_aligned = self.X[i] + X_aligned
            Y_aligned = '_' + Y_aligned
            i-=1

        while j >= 0:
            X_aligned = '_' + X_aligned
            Y_aligned = self.Y[j] + Y_aligned
            j-=1
        
        return X_aligned, Y_aligned

class SequenceAlignmentEfficient():
    
    def __init__(self,X,Y, alpha, delta):
        self.X = X
        self.Y = Y
        self.alpha = alpha
        self.delta = delta
        
    def calculate_alignment_cost(self,X=None,Y=None):
        if X==None and Y==None:
            X = self.X
            Y = self.Y
        len_x = len(X)
        len_y = len(Y)

        dp = [0 for j in range(len_y+1)]

        for j in range(len_y+1):
            dp[j] = j*self.delta

        for i in range(1,len_x+1):
            
            previous = dp[0]
            dp[0] = i* self.delta
            
            for j in range(1,len_y+1):

                matching =  previous + self.alpha[X[i-1]][Y[j-1]]
                mismatch_x = dp[j] + delta
                mismatch_y = dp[j-1] + delta
                previous = dp[j]
                dp[j] = min(matching,mismatch_x,mismatch_y)
        return dp[-1]
    
        
    def divide(self,X,Y):

        mid = len(X) // 2
        x1, x2 = X[:mid] , X[mid:]
        x2_reverse = x2[::-1]
        y_reverse = Y[::-1]
        
        result1 = []
        result2 = []
        
        for i in range(0,len(Y)+1):
            result1.append(self.calculate_alignment_cost(x1, Y[:i]))
        
        for i in range(0,len(Y)+1):
            result2.append(self.calculate_alignment_cost(x2_reverse, y_reverse[:i]))

        result2.reverse()

        min_divide_cost = float("inf")
        divide_split = 0
        
        for i in range(0,len(Y)+1):
            if (result1[i] + result2[i]) < min_divide_cost:
                min_divide_cost = result1[i] + result2[i]
                divide_split = i
                
        return x1,x2,Y[:divide_split],Y[divide_split:]
    
    def divide_and_conquer(self,X,Y):
        if len(X)<=2 or len(Y)<=2:
            seq = SequenceAlignmentBasic(X,Y,self.alpha,self.delta)
            seq.calculate_alignment_cost()
            x_b,y_b = seq.find_alignment()
            return (x_b, y_b)

        x1, x2, y1, y2 = self.divide(X,Y)
        X1,Y1 = self.divide_and_conquer(x1,y1)
        X2,Y2 = self.divide_and_conquer(x2,y2)
        return (X1 + X2, Y1+Y2)
    
    def find_alignment(self):
        complete,X,Y = check_end_case_and_add_gaps(self.X,self.Y)
        if complete:
            return (X,Y)
        else:
            return self.divide_and_conquer(X,Y)
        
        

alpha =  {'A': {'A':0,'C':110,'G':48,'T':94},
          'C': {'A':110,'C':0,'G':118,'T':48},
          'G':  {'A':48,'C':118,'G':0,'T':110},
          'T': {'A':94,'C':48,'G':110,'T':0}
          }

delta = 30

In [4]:
x = 0
while x < 100:
    x += 1
    X = get_random_string(100)
    Y = get_random_string(100)
    sequence_alignment = SequenceAlignmentEfficient(X,Y,alpha,delta)
    cost = sequence_alignment.calculate_alignment_cost()
    X_align, Y_align = sequence_alignment.find_alignment()
    cost1 = compute_cost(X_align, Y_align, alpha, delta)
    cost2 = check_bio_stats(X, Y, alpha)
    if cost != cost1 or cost1 != cost2 or cost!= int(cost2):
        print(X)
        print(Y)
        print('X_align',X_align)
        print('Y_align',Y_align)
        print("Fail ==== "'cost = ',cost,'cost1 = ',cost1,'cost2 = ',int(cost2))
        print(x)
        break
    else:
        print(f"PASS {x}")


PASS 1
PASS 2
PASS 3
PASS 4
PASS 5
PASS 6
PASS 7
PASS 8
PASS 9
PASS 10
PASS 11
PASS 12
PASS 13
PASS 14
PASS 15
PASS 16
PASS 17
PASS 18
PASS 19
PASS 20
PASS 21
PASS 22
PASS 23
PASS 24
PASS 25
PASS 26
PASS 27
PASS 28
PASS 29
PASS 30
PASS 31
PASS 32
PASS 33
PASS 34
PASS 35
PASS 36
PASS 37
PASS 38
PASS 39
PASS 40
PASS 41
PASS 42
PASS 43
PASS 44
PASS 45
PASS 46
PASS 47
PASS 48
PASS 49
PASS 50
PASS 51
PASS 52
PASS 53
PASS 54
PASS 55
PASS 56
PASS 57
PASS 58
PASS 59
PASS 60
PASS 61
PASS 62
PASS 63
PASS 64
PASS 65
PASS 66
PASS 67
PASS 68
PASS 69
PASS 70
PASS 71
PASS 72
PASS 73
PASS 74
PASS 75
PASS 76
PASS 77
PASS 78
PASS 79
PASS 80
PASS 81
PASS 82
PASS 83
PASS 84
PASS 85
PASS 86
PASS 87
PASS 88
PASS 89
PASS 90
PASS 91
PASS 92
PASS 93
PASS 94
PASS 95
PASS 96
PASS 97
PASS 98
PASS 99
PASS 100
